In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import transforms


In [ ]:
!nvidia-smi

Tue Jan 16 12:58:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc


In [ ]:
def train_step(model: torch.nn.Module,
               data_loader : torch.utils.data.DataLoader,
               loss_fn : torch.nn.Module,
               optimizer : torch.optim.Optimizer,
               accuracy_fn,
               device : torch.device
               ):
  """ This is function for training loop. It takes model,data_loader,loss_fn,optimizer,accuracy_fn,device as an input."""
  # Tracking loss and accuracy
  train_loss,train_acc = 0,0

  # puting data into training mode.
  model.train()
  # adding loop to loop through every training batch.
  for batch , (x,y) in enumerate(data_loader):
    # putting data into target device
    x,y = x.to(device),y.to(device)
    # Forward pass (outputs raw logits)
    y_pred = model(x)
    # calculting loss and accuracy per batch.
    loss = loss_fn(y_pred,y)
    train_loss += loss
    # optimizer zero grad.
    optimizer.zero_grad()
    # Back-Propagation
    loss.backward()
    # optimizer step
    optimizer.step()
    # Calculate and accumulate accuracy metric across all batches
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    train_loss = train_loss / len(data_loader)
    train_acc = train_acc / len(data_loader)

  return train_loss, train_acc





In [ ]:
def test_step(model: torch.nn.modules,
               data_loader : torch.utils.data.DataLoader,
               loss_fn : torch.nn.modules,
               accuracy_fn,
               device : torch.device
               ):
  """ This is function for training loop. It takes model,data_loader,loss_fn,optimizer,accuracy_fn,device as an input."""
  # Tracking loss and accuracy
  test_loss,test_acc = 0,0
  # put model in eval mode.
  model.eval()
  # turn on inference mode.
  with torch.inference_mode():
    # Loop through DataLoader batches
    for batch, (x,y) in enumerate(data_loader):
      # putting data into target device
      x,y = x.to(device),y.to(device)
      # forward pass
      test_pred = model(x)
      # calculate loss and accuracy.
      test_loss += loss_fn(test_pred,y).item()
      #print(test_pred.shape(),y.shape())
      test_acc += ((test_pred.argmax(dim=1) == y).sum().item()/len(test_pred))

    test_loss /= len(data_loader)
    test_acc /= len(data_loader)


  #print(f"test loss : {test_loss : 0.5f} | test accuracy: {test_acc: 0.5f}")

  return test_loss,test_acc


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# **Creating a train() funtion**

In [ ]:
# for loading bar....
from tqdm.auto import tqdm

def train(model : nn.modules,
          train_dataloader : torch.utils.data.DataLoader,
          test_dataloader : torch.utils.data.DataLoader,
          loss_fn : torch.nn.modules,
          optimizer : torch.optim.Optimizer,
          epochs : int,
          accuracy_fn,
          device = device
          ):
  """ It is a train() function which can used To train our Model. """
  # create result dictionary for storing results per epoch
  result = {"train_loss" : [],
            "train_acc" : [],
            "test_loss" : [],
            "test_acc" : [],
            }

  # looping through train and test step.
  for epoch in tqdm(range(epochs)):
    train_loss,train_acc = train_step(model,train_dataloader,loss_fn,optimizer,accuracy_fn,device)
    test_loss,test_acc = test_step(model,test_dataloader,loss_fn,accuracy_fn,device)

    # print out what's happening.......
    print(f"train_loss : {train_loss : 0.4f} | test_loss : {test_loss : 0.4f}")

    # Update the result dictionary.
    result["train_loss"].append(train_loss)
    result["train_acc"].append(train_acc)
    result["test_loss"].append(test_loss)
    result["test_acc"].append(test_acc)



  return result


# **Downloading Custom dataset of sushi, pizza,steak from github repo (Use raw link)**

In [ ]:
import requests
import zipfile
import pathlib
from pathlib import Path


In [ ]:
# setting up the path to the data folder

data_path = Path("data/")

img_path = data_path/"pizza_steak_sushi"

# Download the file if the path doesn't exists

In [ ]:
if img_path.is_dir():
  print("already exits")
else:
  img_path.mkdir(parents= True,
                 exist_ok= True)

already exits


**Unzipping File**

In [ ]:
from zipfile import ZipFile
with ZipFile("pizza_steak_sushi.zip", "r") as zip_ref :
  #zip_ref.get_info()
  zip_ref.extractall("data/pizza_steak_sushi")

**Loading data transform them and convert into DataLoader**

In [ ]:
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
# Creating Transformer for transforming data into tensors, same shape and adding some randomess
data_transform = transforms.Compose([transforms.Resize(size = (256,256)),
                                    transforms.RandomHorizontalFlip(p=0.5),
                                    transforms.TrivialAugmentWide(num_magnitude_bins= 10),
                                    transforms.ToTensor()])


**TRANSFORMING......**

In [ ]:
# Transforming image.
train_data = datasets.ImageFolder(root = "data/pizza_steak_sushi/train",
                                        transform= data_transform,
                                        target_transform=None)
test_data = datasets.ImageFolder(root = "data/pizza_steak_sushi/test",
                                        transform= data_transform,
                                        target_transform=None)

**Turn Loaded image into DataLoader**

In [ ]:
import os
train_dataloader = DataLoader(dataset = train_data,
                              batch_size= 32,
                              num_workers=os.cpu_count(),
                              shuffle=True)
test_dataloader = DataLoader(dataset = test_data,
                              batch_size= 32,
                             num_workers= os.cpu_count(),
                              shuffle=False)

In [ ]:
img, label = next(iter(train_dataloader))
img,label

(tensor([[[[0.4039, 0.4431, 0.4588,  ..., 0.0000, 0.0000, 0.0000],
           [0.3961, 0.4431, 0.4627,  ..., 0.0000, 0.0000, 0.0000],
           [0.3882, 0.4431, 0.4667,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.2078, 0.2078, 0.2078,  ..., 0.8314, 0.8235, 0.8157],
           [0.2000, 0.2039, 0.2078,  ..., 0.8235, 0.8196, 0.8157],
           [0.1961, 0.1961, 0.2039,  ..., 0.8196, 0.8118, 0.8118]],
 
          [[0.2706, 0.3020, 0.3137,  ..., 0.0000, 0.0000, 0.0000],
           [0.2627, 0.3020, 0.3176,  ..., 0.0000, 0.0000, 0.0000],
           [0.2549, 0.3020, 0.3216,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0706, 0.0706, 0.0706,  ..., 0.6824, 0.6824, 0.6784],
           [0.0706, 0.0706, 0.0706,  ..., 0.6706, 0.6706, 0.6667],
           [0.0706, 0.0706, 0.0667,  ..., 0.6588, 0.6588, 0.6549]],
 
          [[0.2235, 0.2471, 0.2471,  ..., 0.0000, 0.0000, 0.0000],
           [0.2118, 0.2471, 0.2510,  ..., 0.0000, 0.0000, 0.0000],
           [0.2039, 0.24

# **Replicating TinyVGG architecture from CNN Explainer**

In [ ]:
class CustomTinnyVGG(nn.Module):
  def __init__(self,input_shape : int,
               hidden_units : int,
               output_shape : int
               ) -> None:
    super().__init__()
    self.convblock_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels= hidden_units,
                  kernel_size = 3,
                  stride = 1,
                  padding = 1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels= hidden_units,
                  kernel_size = 3,
                  stride = 1,
                  padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2)
    )

    self.convblock_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels= hidden_units,
                  kernel_size = 3,
                  stride = 1,
                  padding = 1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels= hidden_units,
                  kernel_size = 3,
                  stride = 1,
                  padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2)
    )

    self.Classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = 61440 ,
                  out_features= 3)

    )

  def forward(self, x):
    return self.Classifier(self.convblock_2(self.convblock_1(x)))


In [ ]:
Model_0 = CustomTinnyVGG(input_shape = 3 , #no. of color chanels
                         hidden_units = 15,
                         output_shape = len(train_data.classes)).to(device)

# set up loss funtion and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = Model_0.parameters(),
                             lr = 0.01)

In [ ]:
# start timer
from timeit import default_timer as timer
start_time = timer()

# TRAIN OUR MODEL

model_0_results = train(model = Model_0,
                        train_dataloader= train_dataloader,
                        test_dataloader = test_dataloader,
                        loss_fn = loss_fn,
                        optimizer = optimizer,epochs = 100,
                        accuracy_fn = accuracy_fn,
                        device = device)
end_time = timer()

print(f"Total time taken : {end_time - start_time : 0.3f}")


  0%|          | 0/100 [00:00<?, ?it/s]

train_loss :  0.1526 | test_loss :  1.2765
train_loss :  0.1429 | test_loss :  1.2022
train_loss :  0.1679 | test_loss :  1.0674
train_loss :  0.1662 | test_loss :  1.1002
train_loss :  0.1558 | test_loss :  1.1129
train_loss :  0.1506 | test_loss :  1.1312
train_loss :  0.1465 | test_loss :  1.1561
train_loss :  0.1664 | test_loss :  1.1010
train_loss :  0.1530 | test_loss :  1.1148
train_loss :  0.1604 | test_loss :  1.0983
train_loss :  0.1542 | test_loss :  1.1044
train_loss :  0.1486 | test_loss :  1.1243
train_loss :  0.1485 | test_loss :  1.1477
train_loss :  0.1436 | test_loss :  1.1486
train_loss :  0.1659 | test_loss :  1.0952
train_loss :  0.1561 | test_loss :  1.0785
train_loss :  0.1558 | test_loss :  1.0591
train_loss :  0.1506 | test_loss :  1.1266
train_loss :  0.1649 | test_loss :  1.0600
train_loss :  0.1407 | test_loss :  1.2270
train_loss :  0.1558 | test_loss :  1.0551
train_loss :  0.1668 | test_loss :  0.9854
train_loss :  0.1606 | test_loss :  1.2741
train_loss 

In [ ]:
model_0_results["train_loss"]